In [32]:
import numpy as np
from keras.models import Model
from keras.layers import Input, merge, Convolution3D, MaxPooling3D, UpSampling3D, GlobalAveragePooling3D, Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.optimizers import SGD, Adam, Nadam

from keras_tqdm import TQDMNotebookCallback

In [2]:
X = np.load("/mnt/data/luna16/sets/toy_set_v1_20k.npy")
y = np.zeros((20000,2))
y[:10000,0] = 1
y[10000:,1] = 1

idx = np.random.permutation(X.shape[0])
X, y = X[idx], y[idx]

In [21]:
X = (X + 693.0) / 343.0

In [48]:
w, h, d = 32, 32, 32

alpha = 2

def get_test3d():
    inputs = Input((w, h, d, 1))
    sz = 32

    def conv3dparams():
        return { 'activation':ELU(), 'border_mode':'valid', 'init': 'he_normal' }

    x = Convolution3D(sz, 3, 3, 3, **conv3dparams())(inputs)
    x = BatchNormalization()(x)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams())(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    sz = int(sz * alpha)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams())(x)
    x = BatchNormalization()(x)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams())(x)
    x = BatchNormalization()(x)
    # x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    sz = int(sz * alpha)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams())(x)
    x = BatchNormalization()(x)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams())(x)
    x = BatchNormalization()(x)
    # x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    sz = int(sz * alpha)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams())(x)
    x = BatchNormalization()(x)
    x = Convolution3D(sz, 3, 3, 3, **conv3dparams())(x)
    x = BatchNormalization()(x)
    # x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    sz = int(sz * alpha)
    # x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    # x = Convolution3D(sz, 3, 3, 3, **conv3dparams)(x)
    # x = BatchNormalization()(x)

    x = Convolution3D(sz, 2, 2, 2, activation=ELU(), border_mode='valid')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    # x = GlobalAveragePooling3D()(x)
    x = Dense(2, activation='softmax')(x)

    model = Model(input=inputs, output=x)

    return model

model = get_test3d()
print(model.summary())

#sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam())

batch_size=32

model.fit(
    X, 
    y, 
    batch_size=batch_size,
    nb_epoch=10,
    validation_split=0.1,
    verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True)])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (None, 32, 32, 32, 1) 0                                            
____________________________________________________________________________________________________
convolution3d_90 (Convolution3D) (None, 30, 30, 30, 32 896         input_11[0][0]                   
____________________________________________________________________________________________________
batchnormalization_59 (BatchNorm (None, 30, 30, 30, 32 128         convolution3d_90[0][0]           
____________________________________________________________________________________________________
convolution3d_91 (Convolution3D) (None, 28, 28, 28, 32 27680       batchnormalization_59[0][0]      
___________________________________________________________________________________________

KeyboardInterrupt: 

In [8]:
sgd.lr = 0.0001

model.fit(
    X, 
    y, 
    batch_size=batch_size,
    nb_epoch=3,
    validation_split=0.1,
    verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True)])

In [9]:
sgd.lr = 0.00001

model.fit(
    X, 
    y, 
    batch_size=batch_size,
    nb_epoch=3,
    validation_split=0.1,
    verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True)])

In [16]:
np.mean(X)

-693.14036520690922

In [15]:
np.std(X)

343.60703254609479

In [35]:
y_pred = model.predict(X[18000:], batch_size)

In [46]:
y_true = y[18000:]
idxs = (y_true[:,1] - y_pred[:,1]).argsort()[-30:][::-1]

In [47]:
for i in idxs:
    print(y_true[i], y_pred[i])

[ 0.  1.] [  9.99996543e-01   2.53279723e-06]
[ 0.  1.] [  9.99869943e-01   1.20705336e-04]
[ 0.  1.] [  9.97346520e-01   5.37337095e-04]
[ 0.  1.] [ 0.99783343  0.00127632]
[ 0.  1.] [ 0.99925703  0.00157411]
[ 0.  1.] [ 0.99981576  0.00224341]
[ 0.  1.] [ 0.99868125  0.00632586]
[ 0.  1.] [ 0.99948692  0.00880938]
[ 0.  1.] [ 0.99677938  0.04699558]
[ 0.  1.] [ 0.99800247  0.05455635]
[ 0.  1.] [ 0.99890196  0.05793238]
[ 0.  1.] [  7.28094873e-09   6.92331195e-02]
[ 0.  1.] [  2.06694438e-07   7.26839751e-02]
[ 0.  1.] [  1.99966351e-07   7.31601343e-02]
[ 0.  1.] [  2.37014305e-07   8.01547989e-02]
[ 0.  1.] [  6.03650051e-07   8.16362053e-02]
[ 0.  1.] [  6.28011094e-07   8.44779685e-02]
[ 0.  1.] [ 0.99990225  0.08570059]
[ 0.  1.] [  4.33808424e-07   8.78540277e-02]
[ 0.  1.] [  4.32186397e-07   9.17938799e-02]
[ 0.  1.] [  1.17456672e-07   9.35307071e-02]
[ 0.  1.] [  1.84738226e-07   9.53920409e-02]
[ 0.  1.] [  7.95592427e-07   9.55004245e-02]
[ 0.  1.] [  7.33311765e-07   9.